In [1]:
# Combined 2 fiels from JT
import pandas as pd
import googlemaps
import json
from haversine import haversine
import datetime
import zipcodes
import numpy as np

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [2]:
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Sears'

In [3]:
file_1_closing=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Sears/SHLD Closings.xlsx",dtype=str)
file_2_current=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Sears/SHLD Current Locations.csv",dtype=str)

file_1_closing=file_1_closing.rename(columns={"Latitude":"lat","Longitude":"lng"})
file_2_current=file_2_current.rename(columns={"latitude":"lat","longitude":"lng"})

location_lat_lng=file_1_closing[['lat','lng']].append(file_2_current[['lat','lng']])
location_lat_lng['lat']=location_lat_lng['lat'].astype(float)
location_lat_lng['lng']=location_lat_lng['lng'].astype(float)
location_lat_lng=location_lat_lng.drop_duplicates().reset_index()
del location_lat_lng['index']
location_lat_lng.shape

(862, 2)

In [4]:
BL_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
BL_store_list.head(2)

BL_sales_last_week=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-01-05.csv",dtype=str)
BL_sales_last_week=BL_sales_last_week[BL_sales_last_week['week_end_date']=="2019-01-05"]
BL_sales_last_week['sales']=BL_sales_last_week['sales'].astype(float)
BL_sales_last_week=BL_sales_last_week[BL_sales_last_week['sales']>0]
BL_sales_last_week=BL_sales_last_week[BL_sales_last_week['location_id']!="6990"]
BL_store_list=BL_store_list[BL_store_list['location_id'].isin(BL_sales_last_week['location_id'].tolist())]

BL_new_stores=BL_sales_last_week[~BL_sales_last_week['location_id'].isin(BL_store_list['location_id'].tolist())] # 0, all found

BL_store_list=BL_store_list[['location_id','latitude_meas','longitude_meas']]
BL_store_list['latitude_meas']=BL_store_list['latitude_meas'].astype(float)
BL_store_list['longitude_meas']=BL_store_list['longitude_meas'].astype(float)
BL_store_list=BL_store_list.reset_index()
del BL_store_list['index']

In [5]:
counter_i=0
output_nearest_store=pd.DataFrame()
for i in range(len(location_lat_lng)):
    sears_center=[location_lat_lng['lat'][i],location_lat_lng['lng'][i]]
    dist_0=99999999
    nearest_location="0"
    BL_lat=9999999
    BL_lng=9999999
    for j in range(len(BL_store_list)):
        BL_center=[BL_store_list['latitude_meas'][j],BL_store_list['longitude_meas'][j]]
        dist=haversine(sears_center,BL_center,miles=True)
        if dist<dist_0:
            dist_0=dist
            nearest_location=BL_store_list['location_id'][j]
            BL_lat=BL_center[0]
            BL_lng=BL_center[1]
    counter_i+=1
    df=pd.DataFrame({"Sears_Lat":sears_center[0],"Sears_Lng":sears_center[1],"nearest_BL":nearest_location,"closest_dist_between_BL_Sears":dist_0,"BL_lat":BL_lat,"BL_lng":BL_lng},index=[counter_i])
    output_nearest_store=output_nearest_store.append(df)
      
    
output_nearest_store=output_nearest_store[output_nearest_store['closest_dist_between_BL_Sears']<=20] # Fileter 20 miles nearest BL stores

In [6]:

BL_store_list=output_nearest_store['nearest_BL'].unique().tolist()
len(BL_store_list)

505

In [7]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips=store_level_zips[['location_id','revenue_flag','zip']].rename(columns={"zip":"zip_cd"})
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(['P','S'])]
store_level_zips=store_level_zips[store_level_zips['location_id'].isin(BL_store_list)]
store_level_zips=store_level_zips.reset_index()
del store_level_zips['index']

len(store_level_zips['location_id'].unique())

485

In [8]:
BL_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
BL_store_list['latitude_meas']=BL_store_list['latitude_meas'].astype(float)
BL_store_list['longitude_meas']=BL_store_list['longitude_meas'].astype(float)
new_store=output_nearest_store[~output_nearest_store['nearest_BL'].isin(store_level_zips['location_id'].unique().tolist())]['nearest_BL'].unique().tolist()
BL_store_list_new=BL_store_list[BL_store_list['location_id'].isin(new_store)]

BL_store_list_new=BL_store_list_new[['location_id','latitude_meas','longitude_meas']].reset_index()
del BL_store_list_new['index']

BL_store_list_lat_dict=BL_store_list.set_index(['location_id'])['latitude_meas'].to_dict()
BL_store_list_lng_dict=BL_store_list.set_index(['location_id'])['longitude_meas'].to_dict()

In [9]:
output_nearest_store.head(2)

,BL_lat,BL_lng,Sears_Lat,Sears_Lng,closest_dist_between_BL_Sears,nearest_BL
1,38.813626,-77.134629,38.805997,-76.983372,8.160659,5281
2,30.006953,-90.220381,30.002570,-90.218460,0.323911,4150


In [10]:
df_BL_nearest=output_nearest_store[['nearest_BL','BL_lat','BL_lng']].drop_duplicates().reset_index()
del df_BL_nearest['index']
df_BL_nearest.tail(2)

,nearest_BL,BL_lat,BL_lng
503,4045,34.520068,-117.315364
504,1429,33.942573,-83.448270


In [11]:
df_BL_nearest_avaiable=df_BL_nearest[~df_BL_nearest['nearest_BL'].isin(BL_store_list_new['location_id'].tolist())].reset_index()
df_BL_nearest_new=df_BL_nearest[df_BL_nearest['nearest_BL'].isin(BL_store_list_new['location_id'].tolist())].reset_index()

del df_BL_nearest_avaiable['index']
del df_BL_nearest_new['index']

In [12]:
counter_i=0
df_zip_in_10_miles_available=pd.DataFrame()
for i in range(len(df_BL_nearest_avaiable)):
    location_id=df_BL_nearest_avaiable['nearest_BL'][i]
    store_center=[df_BL_nearest_avaiable['BL_lat'][i],df_BL_nearest_avaiable['BL_lng'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip_cd":zip_cd,"dist_miles":dist,"revenue_flag":"zips_10_for_labeled_store"},index=[counter_i])
            counter_i+=1
            df_zip_in_10_miles_available=df_zip_in_10_miles_available.append(df)
            

In [13]:
counter_i=0
df_zip_in_10_miles_new=pd.DataFrame()
for i in range(len(df_BL_nearest_new)):
    location_id=df_BL_nearest_new['nearest_BL'][i]
    store_center=[df_BL_nearest_new['BL_lat'][i],df_BL_nearest_new['BL_lng'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip_cd":zip_cd,"dist_miles":dist,"revenue_flag":"zips_10_about_new_store"},index=[counter_i])
            counter_i+=1
            df_zip_in_10_miles_new=df_zip_in_10_miles_new.append(df)

            
df_zip_in_10_miles=df_zip_in_10_miles_available.append(df_zip_in_10_miles_new)

In [14]:
store_level_zips['dist_miles']=np.nan
for i in range(len(store_level_zips)):
    store_id=store_level_zips['location_id'][i]
    zip_cd=store_level_zips['zip_cd'][i]
    store_lat=BL_store_list_lat_dict[store_id]
    store_lng=BL_store_list_lng_dict[store_id]
    store_center=[store_lat,store_lng]
    
    if zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
    else:
        try:
            zip_center_zipcodes=[float(zipcodes.matching(zip_cd)['lat']),float(zipcodes.matching(zip_cd)['long'])]
            dist=haversine(zip_center_zipcodes,store_center,miles=True)
        except:
            dist=np.nan
        
    store_level_zips['dist_miles'][i]=dist
    
store_level_zips=store_level_zips[(store_level_zips['revenue_flag']=="P") | (store_level_zips['dist_miles']<=20)]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
zips_for_BL_stores_S_20_miles=store_level_zips.append(df_zip_in_10_miles)
zips_for_BL_stores_S_20_miles=zips_for_BL_stores_S_20_miles.sort_values(['location_id','zip_cd','revenue_flag','dist_miles'])
output_nearest_store=pd.merge(output_nearest_store,zips_for_BL_stores_S_20_miles,left_on="nearest_BL",right_on="location_id",how="left")


In [19]:
zip_HH=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Sears/BL_zips_with_HH_info_25_2019-01-17.xlsx",dtype=str)
zip_HH['HH18']=zip_HH['HH18'].astype(float)
zip_HH=zip_HH[zip_HH['HH18']>=1000]
zip_list_HH_1000P=zip_HH['ZIP_CODE'].unique().tolist()

In [22]:
output_nearest_store_1=output_nearest_store[output_nearest_store['revenue_flag'].isin(['P','S'])]
output_nearest_store_2=output_nearest_store[~output_nearest_store['revenue_flag'].isin(['P','S'])]
output_nearest_store_2=output_nearest_store_2[output_nearest_store_2['zip_cd'].isin(zip_list_HH_1000P)]
output_nearest_store=output_nearest_store_1.append(output_nearest_store_2)

In [23]:
output_unique_zips=output_nearest_store.sort_values(['revenue_flag','zip_cd','dist_miles']).drop_duplicates(['zip_cd'])

In [24]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Sears/BL_zips_for_nearest_BL_stores_to_Sears_Kmart_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_unique_zips.to_excel(writer,"unique_zips",index=False)
zips_for_BL_stores_S_20_miles.to_excel(writer,"zips_by_store",index=False)
file_1_closing.to_excel(writer,"SHLD Closings",index=False)
file_2_current.to_excel(writer,"SHLD Current Locations",index=False)
writer.save()